In [85]:
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import time

In [86]:
# Parameters for Indeed scraper: 
jobTitle = '"Software Engineer"' # Use double quotes inside of single quotes for exact match on the title
location = 'United States'
maxJobs = 200 # Can only grab jobs in increments of 10 for now.
# Going to remove salaryEstimate for now because so many jobs don't list a salary. 
# We can filter by salary after the data is collected.
# salaryEstimate = "50" # This number represents thousands of dollars. i.e. 50 = $50,000

# Target URL Format: https://www.indeed.com/jobs?q=Data+Scientist&l=New+York

def makeURLs(jobTitle, location, maxJobs):
    
    numPages = maxJobs / 10
    numPages = int(numPages)
    
    root = 'http://www.indeed.com/jobs?q='
    
    jobTitle = jobTitle.replace(' ', '+')
    location = location.replace(' ', '+')
    
    URLs = []
    
    pageNum = 0
    for page in range(numPages):
    
        if pageNum == 0:
            seedURL = root + jobTitle + '&l=' + location
        else:
            jobNum = pageNum*10
            jobNum = str(jobNum)
            seedURL = root + jobTitle + '&l=' + location + '&start=' + jobNum
        pageNum = pageNum + 1
        URLs.append(seedURL)
        
    return URLs

seedURLs = makeURLs(jobTitle, location, maxJobs)

In [87]:
#Get URL for every job listed on the page. We'll remove duplicates later.

# Get list of links to Listing

listingURLs = []
    
locations = []
    
for seedURL in seedURLs:
    print(seedURL)

    # Get HTML for each Seed URL to extract links to job listing pages.
    page = requests.get(seedURL)
    time.sleep(1) # One second pause between requests.
    soup = BeautifulSoup(page.text, 'html.parser')

    # Loop through html and append to list all page URLs linked to by job title.
    for div in soup.find_all(name='div', attrs={'class':'row'}):

        # Get Listing URL
        for a in div.find_all(name='a', attrs={'data-tn-element':'jobTitle'}):
            listingURLs.append(a['href'])

#Remove URLs containing the string '/pagead/' or '/company'      
listingURLs = [ x for x in listingURLs if '/pagead/' not in x ]
listingURLs = [ x for x in listingURLs if '/company/' not in x ]

http://www.indeed.com/jobs?q="Software+Engineer"&l=United+States
http://www.indeed.com/jobs?q="Software+Engineer"&l=United+States&start=10
http://www.indeed.com/jobs?q="Software+Engineer"&l=United+States&start=20
http://www.indeed.com/jobs?q="Software+Engineer"&l=United+States&start=30
http://www.indeed.com/jobs?q="Software+Engineer"&l=United+States&start=40
http://www.indeed.com/jobs?q="Software+Engineer"&l=United+States&start=50
http://www.indeed.com/jobs?q="Software+Engineer"&l=United+States&start=60
http://www.indeed.com/jobs?q="Software+Engineer"&l=United+States&start=70
http://www.indeed.com/jobs?q="Software+Engineer"&l=United+States&start=80
http://www.indeed.com/jobs?q="Software+Engineer"&l=United+States&start=90
http://www.indeed.com/jobs?q="Software+Engineer"&l=United+States&start=100
http://www.indeed.com/jobs?q="Software+Engineer"&l=United+States&start=110
http://www.indeed.com/jobs?q="Software+Engineer"&l=United+States&start=120
http://www.indeed.com/jobs?q="Software+Engin

In [91]:
# DataFrame to hold job listing information
df = pd.DataFrame()

# Set column max_colwidth to compare non-truncated URLs
pd.set_option('display.max_colwidth', 500)

# Create DataFrame Series of scraped listing url suffixes
url_suffix = pd.Series(finalURLs)
df['url_suffix'] = url_suffix.values

# Generate Full URLs for each suffix and add it to the dataframe
urls = []
for i, row in df.iterrows():
    urls.append('http://www.indeed.com' + df['url_suffix'][i])
url_full = pd.Series(urls)
df['url_full'] = url_full.values

df.head()

,url_suffix,url_full
0,/rc/clk?jk=acec0008c6542da0&fccid=5a31ef999afce663&vjs=3,http://www.indeed.com/rc/clk?jk=acec0008c6542da0&fccid=5a31ef999afce663&vjs=3
1,/rc/clk?jk=4b9111710c4bc473&fccid=aeb15e43a6800b9d&vjs=3,http://www.indeed.com/rc/clk?jk=4b9111710c4bc473&fccid=aeb15e43a6800b9d&vjs=3
2,/rc/clk?jk=937b1f1f803a9870&fccid=4db2ddf023fb6496&vjs=3,http://www.indeed.com/rc/clk?jk=937b1f1f803a9870&fccid=4db2ddf023fb6496&vjs=3
3,/rc/clk?jk=e1043df92a15df81&fccid=c267f29f0f85e8b8&vjs=3,http://www.indeed.com/rc/clk?jk=e1043df92a15df81&fccid=c267f29f0f85e8b8&vjs=3
4,/rc/clk?jk=e478cbf844c5b390&fccid=37291b9f5627f8c4&vjs=3,http://www.indeed.com/rc/clk?jk=e478cbf844c5b390&fccid=37291b9f5627f8c4&vjs=3


In [92]:
df.shape

(38, 2)

In [74]:
job_titles = []
    
companies = []

locations = []

summaries = []

for url in df['url_full']:

    # Get HTML of the Page
    time.sleep(1)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
           
    # Get Job Title
    for b in soup.find_all(name='b', attrs={'class': 'jobtitle'}):
        job_titles.append(b.text)
    
    # Get Company Name
    for div in soup.find_all(name='div', attrs={'data-tn-component':'jobHeader'}):
        for span in div.find_all(name='span', attrs={'class': 'company'}):
            companies.append(span.text)

    # Get Location from input
    location = soup.find('input', {'id': 'where'}).get('value')
    locations.append(location)

    # Get Summary
    for span in soup.find_all(name='span', attrs={'class': 'summary'}):
        summaries.append(span.text)   
        
print(len(locations))
            
# Lists -> Series to prepare for DataFrame
job_titles = pd.Series(job_titles)
companies = pd.Series(companies)
locations = pd.Series(locations)
summaries = pd.Series(summaries)

# Assign column titles and set values
df['job_title'] = job_titles.values
df['company'] = companies.values
df['location'] = locations.values
df['summary'] = summaries.values

df.head()

http://www.indeed.com/rc/clk?jk=acec0008c6542da0&fccid=5a31ef999afce663&vjs=3
Connecticut
http://www.indeed.com/rc/clk?jk=4b9111710c4bc473&fccid=aeb15e43a6800b9d&vjs=3
Connecticut
http://www.indeed.com/rc/clk?jk=937b1f1f803a9870&fccid=4db2ddf023fb6496&vjs=3
Denver, CO
http://www.indeed.com/rc/clk?jk=e1043df92a15df81&fccid=c267f29f0f85e8b8&vjs=3
Durham, NC
http://www.indeed.com/rc/clk?jk=e478cbf844c5b390&fccid=37291b9f5627f8c4&vjs=3
Seattle, WA
http://www.indeed.com/rc/clk?jk=a034fb2717362300&fccid=31696df7c99497fd&vjs=3
Hollywood, CA
http://www.indeed.com/rc/clk?jk=6421075eb4f6f7bd&fccid=c267f29f0f85e8b8&vjs=3
Santa Clara, CA
http://www.indeed.com/rc/clk?jk=e0959fc96eeca0f3&fccid=8bc01be85b97ce0d&vjs=3
Redmond, WA
http://www.indeed.com/rc/clk?jk=9cbe0fd2cedc9a8f&fccid=3e008ee9930fee16&vjs=3
San Francisco, CA
http://www.indeed.com/rc/clk?jk=a034fb2717362300&fccid=31696df7c99497fd&vjs=3
Hollywood, CA
http://www.indeed.com/rc/clk?jk=3c6543a115cc7d32&fccid=f1374be6a45f4b8a&vjs=3
San Diego,

In [75]:
df.head(100)

# View DataFrame to make sure all of the information is there.
# df.head()

,url_suffix,url_full,job_title,company,location,summary
0,/rc/clk?jk=acec0008c6542da0&fccid=5a31ef999afce663&vjs=3,http://www.indeed.com/rc/clk?jk=acec0008c6542da0&fccid=5a31ef999afce663&vjs=3,Full Stack Software Engineer,Nine Summer,Connecticut,"This position is on the product team responsible for a leading SaaS product that is used by organizations throughout the US and hundreds of thousands of people. We have an office near New Haven, Connecticut - but you can be based anywhere.\n\nOur partner, Patron Technology is a provider of technology solutions for arts and cultural organizations. Its flagship product, PatronManager CRM, is an application built on the Force.com platform that allows organizations to run all aspects of their bu..."
1,/rc/clk?jk=4b9111710c4bc473&fccid=aeb15e43a6800b9d&vjs=3,http://www.indeed.com/rc/clk?jk=4b9111710c4bc473&fccid=aeb15e43a6800b9d&vjs=3,Software Engineer Asc,Lockheed Martin,Connecticut,This Lockheed Martin Enterprise Business Services (EBS) Software Engineer position is a Java / COTS developer supporting the Rotary & Mission Systems (RMS) business area engineering applications.\n\nA developer with a strong technical background is needed to join an existing high performing team supporting business critical applications for the RMS Engineering & Technology organization. Successful candidates will be expected to quickly come up to speed on new technologies and internal proces...
2,/rc/clk?jk=937b1f1f803a9870&fccid=4db2ddf023fb6496&vjs=3,http://www.indeed.com/rc/clk?jk=937b1f1f803a9870&fccid=4db2ddf023fb6496&vjs=3,Senior Software Engineer,Rebric,"Denver, CO","Rebric is looking for a Senior Software Engineer to help shape and build a team that embraces a DevOps Culture. You will have the opportunity to help implement a new microservices based architecture from the ground up, helping make technology choices to allow for future growth and sustainability.\n\nIf you are interested in this position, please fill out this form:\nhttp://rebric.io/applicant/new/4BcBFsFS\nWe will only be considering candidates that complete the survey. We use a machine lear..."
3,/rc/clk?jk=e1043df92a15df81&fccid=c267f29f0f85e8b8&vjs=3,http://www.indeed.com/rc/clk?jk=e1043df92a15df81&fccid=c267f29f0f85e8b8&vjs=3,"Software Engineer, Developer Technology",NVIDIA,"Durham, NC","For two decades, NVIDIA has pioneered visual computing, the art and science of computer graphics. With our invention of the GPU - the engine of modern visual computing - the field has expanded to encompass PC games, movie production, product design, medical diagnosis and scientific research. Today, visual computing is becoming increasingly central to how people interact with technology, and there has never been a more exciting time to join our team.\nWe are now looking for a Software Enginee..."
4,/rc/clk?jk=e478cbf844c5b390&fccid=37291b9f5627f8c4&vjs=3,http://www.indeed.com/rc/clk?jk=e478cbf844c5b390&fccid=37291b9f5627f8c4&vjs=3,Software Engineer (Junior),SkyKick,"Seattle, WA","-------------\nAbout SkyKick\n-------------\n\nSkyKick is a fast-growing pioneer in the cloud market and has won numerous awards from Microsoft Partner of the Year to Red Herring Top 100 North America. Our mission is to build SaaS products that solve the IT industry's cloud challenges. Our products are 100% partner channel focused and have helped over 5,000 global partners build successful cloud businesses by making it easy to move, backup, and manage their customers in the cloud. We empower..."
5,/rc/clk?jk=a034fb2717362300&fccid=31696df7c99497fd&vjs=3,http://www.indeed.com/rc/clk?jk=a034fb2717362300&fccid=31696df7c99497fd&vjs=3,Software Engineer - Java,Ticketmaster,"Hollywood, CA","Job Summary:\nWho we are.\nWe’re fans who help fans everywhere get in to the live events they love. A proud division of Live Nation Entertainment, Ticketmaster has changed the way the world connects with their favorite artists, teams, and shows, and we continue to shape innovation every day. We’re n

In [29]:
# Export to CSV
df.to_csv('test.csv')